# Instruction Fine-Tune Mistral 7B using LoRA
We use [unnatural-instructions](https://github.com/orhonovich/unnatural-instructions) to perform instruction fune-tuning on mistral 7B base model.

In [68]:
from datasets import load_dataset, Dataset, DatasetDict
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig, TrainingArguments, Trainer, BitsAndBytesConfig
import torch
import time
import evaluate
import pandas as pd
import numpy as np

#### Load Mistral 7B

In [69]:
model_name = "mistralai/Mistral-7B-v0.1"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

In [70]:
original_model = AutoModelForCausalLM.from_pretrained(model_name, quantization_config=bnb_config) # .to(device)
tokenizer = AutoTokenizer.from_pretrained(model_name, model_max_length=512, padding_side="left", add_eos_token=True)

tokenizer.pad_token = tokenizer.eos_token

Loading checkpoint shards: 100%|██████████| 2/2 [00:10<00:00,  5.14s/it]


Print number of trainable model parameters

In [71]:
def print_number_of_trainable_model_parameters(model):
    trainable_model_params = 0
    all_model_params = 0
    for _, param in model.named_parameters():
        all_model_params += param.numel()
        if param.requires_grad:
            trainable_model_params += param.numel()
    return f"trainable model parameters: {trainable_model_params}\nall model parameters: {all_model_params}\npercentage of trainable model parameters: {100 * trainable_model_params / all_model_params:.2f}%"

print(print_number_of_trainable_model_parameters(original_model))

trainable model parameters: 262410240
all model parameters: 3752071168
percentage of trainable model parameters: 6.99%


#### Load Unnatural Instruction Dataset through Huggingface

In [92]:
huggingface_dataset_name = "mrm8488/unnatural-instructions-core"

dataset = load_dataset(huggingface_dataset_name)

dataset

DatasetDict({
    train: Dataset({
        features: ['instruction', 'instances'],
        num_rows: 66010
    })
})

Explode 'instances' into separate rows and make sure exploded_dataset remains the same schema as the original dataset

In [108]:
new_datasets = {}

for split, ds in dataset.items():
    # Prepare lists to hold the exploded rows
    exploded_instructions, exploded_instances = [], []

    # Iterate over each row in the dataset
    for row in ds:
        instruction = row['instruction']
        for instance in row['instances']:
            # For each instance, create a new row with the same instruction
            exploded_instructions.append(instruction)
            exploded_instances.append([instance])
    
    # Create a new dataset from the exploded rows
    exploded_data = {'instruction': exploded_instructions, 'instances': exploded_instances}
    new_datasets[split] = Dataset.from_dict(exploded_data)

# Combine the new datasets into a DatasetDict
exploded_dataset = DatasetDict(new_datasets)

exploded_dataset

DatasetDict({
    train: Dataset({
        features: ['instruction', 'instances'],
        num_rows: 68478
    })
})

In [109]:
# Check each instance format
exploded_dataset['train'][0]


{'instruction': 'You will be given a series of words. Output these words in reverse order, with each word on its own line.',
 'instances': [{'constraints': 'None.',
   'input': "Words: ['Hello', 'world'].",
   'instruction_with_input': "You will be given a series of words. Output these words in reverse order, with each word on its own line.\nWords: ['Hello', 'world'].",
   'output': 'world\nHello'}]}

In [110]:
# Sanity check to make sure that each row has only one instance
sanity_df = exploded_dataset['train'].to_pandas()

sanity_df['instances_length'] = sanity_df['instances'].apply(len)
assert sanity_df[sanity_df['instances_length'] != 1].empty

Split data into train, val, and test

In [111]:
# Split the dataset into train, test, and validation sets
train_test_dataset = exploded_dataset['train'].train_test_split(test_size=0.2, seed=42)  # 80% train, 20% for test and validation
test_val_dataset = train_test_dataset['test'].train_test_split(test_size=0.5, seed=42)  # Split the 20% equally into test and validation

# Create a new DatasetDict
dataset_dict = DatasetDict({
    'train': train_test_dataset['train'],
    'test': test_val_dataset['train'],
    'validation': test_val_dataset['test']
})

dataset_dict

DatasetDict({
    train: Dataset({
        features: ['instruction', 'instances'],
        num_rows: 54782
    })
    test: Dataset({
        features: ['instruction', 'instances'],
        num_rows: 6848
    })
    validation: Dataset({
        features: ['instruction', 'instances'],
        num_rows: 6848
    })
})

In [112]:
index = 200

print(f"instructions: {dataset_dict['test'][index]['instruction']}")
print(f"instances: {dataset_dict['test'][index]['instances']}")

instructions: You are given a passage with certain words/phrases bolded. Identify if the word/phrase is being used in context ofSTAGE DIRECTIONS orCHARACTERIZATION.
instances: [{'constraints': 'The output for each word should be either 0 (Stage Direction) or 1 (Characterization).', 'input': 'STAGE DIRECTIONS:As he enters,MACBETH sees the three witches.He stopshis steps CHARACTERIZATION: amazed and terrified at their sight.', 'instruction_with_input': 'You are given a passage with certain words/phrases bolded. Identify if the word/phrase is being used in context ofSTAGE DIRECTIONS orCHARACTERIZATION.\nSTAGE DIRECTIONS:As he enters,MACBETH sees the three witches.He stopshis steps CHARACTERIZATION: amazed and terrified at their sight.', 'output': '0\n1\n0\n1'}]


In [113]:
# def get_prompt_output(row):
#     # print(row)
#     inst_dict_list = row['instances']
#     print(inst_dict_list, type(inst_dict_list))
#     # Ignore input for now "input: {inst_dict['input']},\n"
#     input_text_list = [f"instruction_with_input: {inst_dict['instruction_with_input']},\nconstraints: {inst_dict['constraints']}"f"input: {inst_dict['input']},\ninstruction_with_input: {inst_dict['instruction_with_input']},\nconstraints: {inst_dict['constraints']}"
#                   for inst_dict in inst_dict_list]
#     input_text = "\n".join(input_text_list)
#     prompt = f"<s>[INST] {input_text} [/INST]"

    
#     output_list = [inst_dict['output'] for inst_dict in inst_dict_list]
#     output = "\n".join(output_list)

#     return prompt, output

# get_prompt_output(dataset_dict['test'][index])

### Perform Fine-Tuning

#### Preprocess Dataset to Generate Prompt|

In [120]:
def get_prompt(d_li):
    p_li  = []
    for d in d_li:
        constraints = f"\n{d['constraints']}" if d['constraints'] else ""
        instruction = f"{d['instruction_with_input']}{constraints}"
        model_answer = d['output']

        prompt = f"<s>[INST] {instruction} [/INST] {model_answer}</s>"
        p_li.append(prompt)
    return "\n".join(p_li)
    # instruction_with_input = f"instruction_with_input: {d['instruction_with_input']}"
    # constraints = f"constraints: {d['constraints']}"
    # return f"<s>[INST] {instruction_with_input},\n{constraints} [/INST]"

def get_output(d_li):
    return "\n".join([d['output'] for d in d_li])

def tokenize_function(example):
    prompts = [get_prompt(per_inst_d_li) for per_inst_d_li in example['instances']]
    outputs = [get_output(per_inst_d_li) for per_inst_d_li in example['instances']]
    # raw_p, out = get_prompt_output(example)
    # prompt, output = raw_p, out
    example['input_ids'] = tokenizer(prompts, max_length=512, padding="max_length", truncation=True, return_tensors="pt").input_ids # .to(device)
    example['labels'] = tokenizer(outputs, max_length=512, padding="max_length", truncation=True, return_tensors="pt").input_ids
    return example

tokenized_datasets = dataset_dict.map(tokenize_function, batched=True, remove_columns=['instruction', 'instances']) # 

tokenized_datasets

Map: 100%|██████████| 6848/6848 [00:01<00:00, 4508.05 examples/s]


DatasetDict({
    train: Dataset({
        features: ['input_ids', 'labels'],
        num_rows: 54782
    })
    test: Dataset({
        features: ['input_ids', 'labels'],
        num_rows: 6848
    })
    validation: Dataset({
        features: ['input_ids', 'labels'],
        num_rows: 6848
    })
})

In [121]:
# To save some time in the lab, you will subsample the dataset:
subsampled_tokenized_datasets = tokenized_datasets.filter(lambda example, index: index % 100 == 0, with_indices=True)

subsampled_tokenized_datasets

Filter: 100%|██████████| 6848/6848 [00:01<00:00, 5192.75 examples/s]


In [122]:
print(f"Shapes of the datasets:")
print(f"Training: {tokenized_datasets['train'].shape}")
print(f"Validation: {tokenized_datasets['validation'].shape}")
print(f"Test: {tokenized_datasets['test'].shape}")

print(tokenized_datasets)

Shapes of the datasets:
Training: (54782, 2)
Validation: (6848, 2)
Test: (6848, 2)
DatasetDict({
    train: Dataset({
        features: ['input_ids', 'labels'],
        num_rows: 54782
    })
    test: Dataset({
        features: ['input_ids', 'labels'],
        num_rows: 6848
    })
    validation: Dataset({
        features: ['input_ids', 'labels'],
        num_rows: 6848
    })
})


In [124]:
print(f"input len is {len(tokenized_datasets['train'][4]['input_ids'])}")
print(f"input Ids: {tokenized_datasets['train'][0]['input_ids']}")
print(f"labels: {tokenized_datasets['train'][0]['labels']}")

input len is 512
input Ids: [2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,

### Fin-Tune the Model with Preprocessed Dataset
(Warning) It will OOM on my machine, so I marked them out

In [125]:
# output_dir = f'./model/dialogue-summary-training-{str(int(time.time()))}'

# training_args = TrainingArguments(
#     output_dir=output_dir,
#     learning_rate=1e-5,
#     per_device_train_batch_size=2,
#     num_train_epochs=1,
#     weight_decay=0.01,
#     logging_steps=1,
#     max_steps=1,
#     fp16=True,
# )

# trainer = Trainer(
#     model=original_model,
#     args=training_args,
#     train_dataset=tokenized_datasets['train'],
#     eval_dataset=tokenized_datasets['validation'],
# )

In [126]:
# trainer.train() ### OOM

### Peft

#### Set up LoRA

In [127]:
from peft import prepare_model_for_kbit_training

original_model.enable_input_require_grads()
original_model.gradient_checkpointing_enable()
original_model = prepare_model_for_kbit_training(original_model)

print(print_number_of_trainable_model_parameters(original_model))

trainable model parameters: 0
all model parameters: 3752071168
percentage of trainable model parameters: 0.00%


In [128]:
from peft import LoraConfig, get_peft_model

lora_config = LoraConfig(
    r=8, # Rank
    lora_alpha=16,
    target_modules=[
        "q_proj",
        "k_proj",
        "v_proj",
        "o_proj",
        "gate_proj",
        "up_proj",
        "down_proj",
        "lm_head",
    ],
    lora_dropout=0.05,
    bias="none",
    task_type='CAUSAL_LM'
)

In [129]:
peft_model = get_peft_model(original_model, 
                            lora_config)
print(print_number_of_trainable_model_parameters(peft_model))

trainable model parameters: 21260288
all model parameters: 3773331456
percentage of trainable model parameters: 0.56%


In [130]:
output_dir = f'./peft-causal-lm-training-{str(int(time.time()))}'

peft_training_args = TrainingArguments(
    output_dir=output_dir,
    auto_find_batch_size=True,
    learning_rate=1e-3, # Higher learning rate than full fine-tuning.
    num_train_epochs=1,
    logging_steps=1,
    max_steps=1    
)
    
peft_trainer = Trainer(
    model=peft_model,
    args=peft_training_args,
    train_dataset=tokenized_datasets["train"],
)

In [131]:
peft_trainer.train()

/home/iris/anaconda3/envs/mistral_ai_gpt/lib/python3.9/site-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss
1,12.883500


TrainOutput(global_step=1, training_loss=12.883462905883789, metrics={'train_runtime': 11.1972, 'train_samples_per_second': 0.714, 'train_steps_per_second': 0.089, 'total_flos': 175274075357184.0, 'train_loss': 12.883462905883789, 'epoch': 0.0})

In [132]:
peft_model_path="./peft-dialogue-summary-checkpoint-local"

peft_trainer.model.save_pretrained(peft_model_path)
tokenizer.save_pretrained(peft_model_path)

/home/iris/anaconda3/envs/mistral_ai_gpt/lib/python3.9/site-packages/peft/utils/save_and_load.py:131: UserWarning: Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.
  warnings.warn("Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.")


('./peft-dialogue-summary-checkpoint-local/tokenizer_config.json',
 './peft-dialogue-summary-checkpoint-local/special_tokens_map.json',
 './peft-dialogue-summary-checkpoint-local/tokenizer.model',
 './peft-dialogue-summary-checkpoint-local/added_tokens.json',
 './peft-dialogue-summary-checkpoint-local/tokenizer.json')

In [133]:
test = dataset_dict['test'][0:2]['instances']
[get_prompt(per_inst_d_li) for per_inst_d_li in test]

['<s>[INST] You are given a list of movie ratings. Each rating consists of a movie title and the corresponding rating out of 5 stars. The task is to find all movies with a rating greater than or equal to 4 stars and print them in alphabetical order.\nAnt-Man - 4stars Captain America: Civil War - 5stars Deadpool - 3stars Guardians of the Galaxy Vol. 2 - 5stars Logan - 4stars Spider-Man: Homecoming - 5stars Thor Ragnarok - 4stars.\nPrint each movie title on a new line in alphabetical order. [/INST] Ant-Man\nCaptain America: Civil War\nGuardians of the Galaxy Vol. 2\nLogan\nSpider-Man: Homecoming\nThor Ragnarok</s>',
 "<s>[INST] In this task, you are asked to summarize a text. The input is one paragraph containing up to 300 words. Your job is to return the top 5 most important sentences from the given text as a list in order of importance.\nI have three sisters and two brothers. We are all close in age, so we are very close growing up. My favorite sister is the middle one. She was always 

In [ ]:
# model_input = tokenizer(eval_prompt, return_tensors="pt").to("cuda")

# ft_model.eval()
# with torch.no_grad():
#     print(eval_tokenizer.decode(ft_model.generate(**model_input, max_new_tokens=100)[0], skip_special_tokens=True))


In [134]:
eval_instances = dataset_dict['test'][0:10]['instances']
prompts = [get_prompt(per_inst_d_li) for per_inst_d_li in eval_instances]
outputs = [get_output(per_inst_d_li) for per_inst_d_li in eval_instances]
#print(prompts)
print(len(prompts), len(outputs), outputs[0:2])


original_model_summaries = []
instruct_model_summaries = []
peft_model_summaries = []

for idx, prompt in enumerate(prompts):
#     prompt = f"""
# Summarize the following conversation.

# {dialogue}

# Summary: """
    
    model_input = tokenizer(prompt, return_tensors="pt")

    original_model_outputs = original_model.generate(**model_input, max_new_tokens=100)
    original_model_text_output = tokenizer.decode(original_model_outputs[0], skip_special_tokens=True)

    peft_model_outputs = peft_model.generate(**model_input, max_new_tokens=100)
    peft_model_text_output = tokenizer.decode(peft_model_outputs[0], skip_special_tokens=True)

    original_model_summaries.append(original_model_text_output)
    peft_model_summaries.append(peft_model_text_output)

zipped_summaries = list(zip(original_model_summaries, peft_model_summaries))
 
df = pd.DataFrame(zipped_summaries, columns = ['original_model_summaries', 'peft_model_summaries'])
df

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


10 10 ['Ant-Man\nCaptain America: Civil War\nGuardians of the Galaxy Vol. 2\nLogan\nSpider-Man: Homecoming\nThor Ragnarok', "['My favorite sister is the middle one.',\n 'She was always so kind and caring, even when we were fighting over who got which toy growing up.',\n 'Now that we are older, she's still my best friend even though we live in different states.',\n 'I talk to her every day and visit her whenever I can.']"]


/home/iris/anaconda3/envs/mistral_ai_gpt/lib/python3.9/site-packages/transformers/generation/utils.py:1636: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(
/home/iris/anaconda3/envs/mistral_ai_gpt/lib/python3.9/site-packages/torch/utils/checkpoint.py:61: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
Setting `pad_token_id` to `eos_token_id`:2 for 

,original_model_summaries,peft_model_summaries
0,[INST] You are given a list of movie ratings. ...,[INST] You are given a list of movie ratings. ...
1,"[INST] In this task, you are asked to summariz...","[INST] In this task, you are asked to summariz..."
2,"[INST] In this task, you are given a natural l...","[INST] In this task, you are given a natural l..."
3,"[INST] In this task, you are given an incomple...","[INST] In this task, you are given an incomple..."
4,[INST] Given a letter and an encrypted message...,[INST] Given a letter and an encrypted message...
5,[INST] It is common for people to use social n...,[INST] It is common for people to use social n...
6,"[INST] In this task, you are given three image...","[INST] In this task, you are given three image..."
7,[INST] You are provided with a description of ...,[INST] You are provided with a description of ...
8,"[INST] In this task, you are given a list of s...","[INST] In this task, you are given a list of s..."
9,"[INST] In this task, you will be given a piece...","[INST] In this task, you will be given a piece..."


In [135]:
print(df['original_model_summaries'].iloc[0])

[INST] You are given a list of movie ratings. Each rating consists of a movie title and the corresponding rating out of 5 stars. The task is to find all movies with a rating greater than or equal to 4 stars and print them in alphabetical order.
Ant-Man - 4stars Captain America: Civil War - 5stars Deadpool - 3stars Guardians of the Galaxy Vol. 2 - 5stars Logan - 4stars Spider-Man: Homecoming - 5stars Thor Ragnarok - 4stars.
Print each movie title on a new line in alphabetical order. [/INST] Ant-Man
Captain America: Civil War
Guardians of the Galaxy Vol. 2
Logan
Spider-Man: Homecoming
Thor Ragnarok


In [136]:
print(df['peft_model_summaries'].iloc[0])

[INST] You are given a list of movie ratings. Each rating consists of a movie title and the corresponding rating out of 5 stars. The task is to find all movies with a rating greater than or equal to 4 stars and print them in alphabetical order.
Ant-Man - 4stars Captain America: Civil War - 5stars Deadpool - 3stars Guardians of the Galaxy Vol. 2 - 5stars Logan - 4stars Spider-Man: Homecoming - 5stars Thor Ragnarok - 4stars.
Print each movie title on a new line in alphabetical order. [/INST] Ant-Man
Captain America: Civil War
Guardians of the Galaxy Vol. 2
Logan
Spider-Man: Homecoming
Thor Ragnarok
